In [1]:
import QuantLib as ql
evDate = ql.Date(11, 7, 2011)
settings = ql.Settings.instance()
settings.setEvaluationDate(evDate)
DC30360 = ql.Thirty360()
Cal = ql.NullCalendar()
BDC = ql.Unadjusted
DGR = ql.DateGeneration.Backward

startDate = ql.Date(13, 7, 2011)
maturityDate = ql.Date(13, 7, 2013)
period = ql.Period(ql.Annual)
sch = ql.Schedule(startDate, maturityDate, period, Cal, BDC, BDC, DGR, False)

TWD = ql.TWDCurrency()
settlementDays = 2
fixingDays = 2

r = 0.05
faceAmt = 100.0
flatTS = ql.FlatForward(0, Cal, r, DC30360, ql.Compounded, ql.Annual)
h_flatTS = ql.YieldTermStructureHandle(flatTS)
Idx12M = ql.IborIndex("Index", period, fixingDays, TWD, Cal, BDC, False, DC30360, h_flatTS)

In [2]:
d0 = ql.Date(13, 7, 2011)
d1 = ql.Date(13, 7, 2012)
d2 = ql.Date(13, 7, 2013)
coupon = 0.05
FixedCP1 = ql.FixedRateCoupon(d1, 100.0, 0.05, DC30360, d0, d1)
FixedCP2 = ql.FixedRateCoupon(d2, 100.0, 0.05, DC30360, d1, d2)
FixedNPA = ql.SimpleCashFlow(100.0, d2)

In [14]:
print(FixedCP1.amount())
print(FixedCP2.amount())

5.000000000000004
5.000000000000004


In [15]:
FixedLeg = ql.Leg([FixedCP1, FixedCP2, FixedNPA])

In [16]:
FloatCP1 = ql.IborCoupon(d1, faceAmt, d0, d1, fixingDays, Idx12M)
FloatCP2 = ql.IborCoupon(d2, faceAmt, d1, d2, fixingDays, Idx12M)
FloatNPA = ql.SimpleCashFlow(faceAmt, d2)

In [17]:
FloatLeg = ql.Leg([FloatCP1, FloatCP2, FloatNPA])

In [18]:
DCVol = ql.Actual365Fixed()
ConstOptTS = ql.ConstantOptionletVolatility(evDate, Cal, BDC, 0.0, DCVol)
h_ConstOptTS = ql.OptionletVolatilityStructureHandle(ConstOptTS)
IborPricer = ql.BlackIborCouponPricer(h_ConstOptTS)

In [19]:
FloatCP1.setPricer(IborPricer)
FloatCP2.setPricer(IborPricer)

In [20]:
Swap = ql.Swap(FixedLeg, FloatLeg)
SwapEngine = ql.DiscountingSwapEngine(h_flatTS)
Swap.setPricingEngine(SwapEngine)

In [21]:
Value = Swap.NPV()
print(Value)

0.0


In [22]:
print(Swap.legNPV(0))
print(Swap.legNPV(1))

-99.97289802648774
99.97289802648774
